In [12]:
import os

from os.path import isfile, join, isdir, dirname, abspath
from os import listdir
import gpxpy
from slugify import slugify


def parseActivityDataInGpxData(gpx =  None):
    if not gpx:
        return None

    
    activity = {}
    activity['time'] = gpx.time
    activity['link'] = gpx.link
    tracks = []
    for track_item in gpx.tracks:
        track = {}
        
        track['name'] = track_item.name
        track['type'] = track_item.type
        segments = []
        for segment_item in track_item.segments:
            segment = {}
            points = []
            for point_item in segment_item.points:
                point = []
                #print(point_item)
                #point.append(point.time)
                point.append(float(point_item.latitude))
                point.append(float(point_item.longitude))
                point.append(float(point_item.elevation))
                points.append(point)
            segment['points'] = points
            segments.append(segment)
        track['segments'] = segments
        tracks.append(track)
    activity['tracks'] = tracks
    
    return activity
    

def getActivityDataFromGpxFile(file_path):
    if not isfile(file_path):
        return None
    
    #print('file_path', file_path)
    file_handle = open(file_path, "r")
    gpx_data = gpxpy.parse(file_handle)
    file_handle.close()
    
    activity_data = parseActivityDataInGpxData(gpx_data)
    
    return activity_data



def getListOfAllFilesInThisDir(dir_path = ''):
    if not dir_path:
        return None
    
    if not isdir(dir_path):
        return None
    
    list_of_all_files = []
    
    for file in listdir(dir_path):
        if not isfile(join(dir_path,file)):
            continue
        #list_of_all_files.append(join(dir_path,file))
        list_of_all_files.append(abspath(join(dir_path,file)))
    
    return list_of_all_files



def getParentDirsAndFilenameFromFullPath(file_path = ''):
    if not file_path:
        return file_path, None
    
    file_path_parts = file_path.split('/')
    
    if len(file_path_parts) <= 1:
        return file_path, None
    
    file_name = file_path_parts[-1]
    
    parent_dirs = file_path.replace(file_name, '')
    
    return file_name, parent_dirs

def encodeStrFromToEncoding(input_str = '', input_encode = 'latin1', output_encode = 'utf-8'):
    output = input_str.encode(input_encode).decode(output_encode)
    
    return output



def renameFileGpx(file_path = '', time = '', link = '', name=''):
    if not file_path:
        return file_path
    
    if not isfile(file_path):
        return file_path
    
    #print('file_path', file_path)
    
    file_name, parent_dirs = getParentDirsAndFilenameFromFullPath(file_path)
    
    if 'activity' in file_name or 'route' in file_name:
        return parent_dirs, file_name
        
    slug_time = slugify(str(time))
    slug_name = slugify(name)
    #print('time', slug_time, 'link', link, 'name', slug_name)
    
    upd_filename = ''
    
    strava_id = ''
    
    if time:
        upd_filename = 'activity'
        parts_filename = file_name.split('.')
        #print('parts_filename[0]', parts_filename[0])
        if parts_filename[0]:
            strava_id = parts_filename[0]
    else:
        upd_filename = 'route'
        # get id from link
        if link:
            if 'routes' in link:
                link_parts = link.split('/')
                if link_parts[-1]:
                    strava_id = link_parts[-1]
        
        if 'last' in slug_name or 'past' in slug_name:
            upd_filename = 'past-activity'
    
    if strava_id:
        upd_filename += '-' + strava_id
    
    if time:
        slug_time = slugify(str(slug_time.split('-')[:3]))
        upd_filename += '-' + slug_time
    
    if name:
        
        upd_filename += '-' + slug_name
    
    
        
    
    
    upd_filename = upd_filename.replace('activity-activity','activity')
    upd_filename = upd_filename.replace('activity-activity','activity')
    
    print(upd_filename)
    
    upd_full_path = parent_dirs + upd_filename + '.gpx'
    
    
    
    
    #print(file_path)
    #print(upd_full_path)
    #print('\n')
    
    os.rename(file_path,upd_full_path) 
    
    return parent_dirs, upd_filename
    
    

def getDataFromFileOfThisDir(dir_path = ''):
    activities = []
    
    list_files = getListOfAllFilesInThisDir(dir_path)
    
    # Pop file without '.gpx'
    index_of_file = 0
    for file_name in list_files:        
        if not file_name[-4:] == '.gpx':
            list_files.pop(index_of_file)
        index_of_file += 1

    data_activities = []
    for gpx_file_path in list_files:
        data_activity = getActivityDataFromGpxFile(gpx_file_path)
        
        parent_dirs, upd_filename = renameFileGpx(
            gpx_file_path, 
            data_activity['time'],
            data_activity['link'],
            data_activity['tracks'][0]['name'],
        )
        
        data_activity['file'] = parent_dirs + upd_filename
        data_activity['file_name'] = upd_filename
        
        if not data_activity:
            continue
        data_activities.append(data_activity)
        
    return data_activities


            
            

dir_gpx_files = 'activities'

data = getDataFromFileOfThisDir(dir_gpx_files)

print(len(data))


route-24240798-dorokhovo-zvenigorod
111


In [13]:
import math

# Limit data up to maxPoints = 998.
# For Yandex map api

def compress_data(data, maxPoints = 998):
    print('start')
    for activity in data:
        for track in activity['tracks']:
            for segment in track['segments']:            
                if len(segment['points']) < maxPoints:
                    continue

                p = segment['points']

                lenpoints = len(p)

                trsh = 1e-12

                def makeTreshold(val):
                    stp = 0.1

                    val = 1
                    for i in range(degs):
                        val *= stp

                    return val

                while(len(p) > maxPoints):
                    n = -1
                    while n < lenpoints - 2:
                        n += 1
                        #print(n, lenpoints)
                        if n == 0 or n == 1:
                            continue;

                        def metr(p1,p2):
                            return (p1[0] - p2[0]) * (p1[0] - p2[0]) + (p1[1] - p2[1]) * (p1[1] - p2[1])

                        trLong = metr(p[n-2],p[n])
                        trSum = metr(p[n-2],p[n-1]) + metr(p[n-1],p[n])
                        trDifr = math.fabs(trSum - trLong)
                        #if trDifr < 0:
                        #    print('!\tMUNUS ')
                        trSmall = metr(p[n-1],p[n])

                        if trDifr < trsh:
                            p.pop(n)
                            lenpoints -= 1
                            n -= 1     

                    trsh = trsh * 1.4
                print(trsh, len(p))

    return data


data = compress_data(data)

start
3.7649707413139464e-06 841
1.3999999999999999e-12 998
1.9209034394458913e-06 991
2.6892648152242475e-06 972
5.270959037839524e-06 988
1.8222555617218566e-07 927
1.9209034394458913e-06 979
1.3720738853184937e-06 990
1.9209034394458913e-06 886
3.7649707413139464e-06 987
2.551157786410599e-07 990
1.9209034394458913e-06 917
1.8222555617218566e-07 987
5.000269261364774e-07 914
1.9209034394458913e-06 936
5.270959037839524e-06 846
2.4201432355484552e-08 942
9.800527752274955e-07 891
3.7649707413139464e-06 852
1.9209034394458913e-06 933
3.7649707413139464e-06 981
9.800527752274955e-07 994
5.000269261364774e-07 911
1.8222555617218566e-07 903
2.551157786410599e-07 995
7.000376965910683e-07 952
3.571620900974838e-07 917
2.6892648152242475e-06 970
9.800527752274955e-07 981
9.800527752274955e-07 982
1.9209034394458913e-06 891
5.270959037839524e-06 963
9.297222253682944e-08 882
5.000269261364774e-07 989
1.3720738853184937e-06 900
5.000269261364774e-07 898
7.000376965910683e-07 988
5.0002692613

In [14]:
# addPastCurrentFutureStatusToData

from datetime import datetime
from slugify import slugify

def getActivityState(name, time, threshold_current_year = 2020):
    status = 'past_activity'
    
    if 'last' in name or 'past' in name:
        return 'past_route'
    
    if not time:
        return 'future'
    
    if int(time.year) == threshold_current_year:
        return 'current'
    
    return status

def getLinkOfActivity(full_file_name):
    file_name = full_file_name.split('/')[-1]
    file_name = file_name.replace('.gpx' , '')
    file_name = slugify(file_name)
    
    file_name = file_name.replace('activity-', '')
    file_name = file_name.split('-')
    file_name = file_name[0]
    print('file_name', file_name)
    
    # https://www.strava.com/activities/200 851 370 5
    if len(file_name) < 6 or len(file_name) > 13:
        return None
    
    #print(file_name)
    link = 'https://www.strava.com/activities/' + file_name
    
    return link
    
    

def addPastCurrentFutureStatusToData(data):
    for index in range(len(data)):
        threshold_year = int(datetime.now().year)
        status = getActivityState(data[index]['tracks'][0]['name'], data[index]['time'], threshold_year)    
        
        #print(status, data[index]['tracks'][0]['name'])
        
        data[index]['state'] = status
        print(data[index]['link'])
        
        if not data[index]['link']:
            link = getLinkOfActivity(data[index]['file_name'])
            data[index]['link'] = link
        
        #print('data[index][state]', data[index]['state'])
          
    return data

data = addPastCurrentFutureStatusToData(data)




https://www.strava.com/routes/22869014
https://www.strava.com/routes/23735851
https://www.strava.com/routes/21875514
None
file_name 2702598567
https://www.strava.com/routes/17337486
https://www.strava.com/routes/17329026
None
file_name 1197883696
https://www.strava.com/routes/17409938
https://www.strava.com/routes/15129981
https://www.strava.com/routes/21891701
None
file_name 2609640020
https://www.strava.com/routes/15130998
https://www.strava.com/routes/16639609
https://www.strava.com/routes/23735746
https://www.strava.com/routes/15163010
None
file_name 2591109123
https://www.strava.com/routes/15130573
https://www.strava.com/routes/15163464
https://www.strava.com/routes/23766746
None
file_name 1674710065
None
file_name 1802901472
https://www.strava.com/routes/16639737
https://www.strava.com/routes/21891487
None
file_name 2668267617
https://www.strava.com/routes/15130089
https://www.strava.com/routes/16639672
None
file_name 1746964989
https://www.strava.com/routes/15129947
https://www.

In [10]:
# Sort data by 

def sort_file_name(activity):
    return activity['file_name']

def sortDataByFileName(input_data):
    data_sort = input_data.copy()
    data_sort.sort(key=sort_file_name,reverse = True)
    return data_sort

data = sortDataByFileName(data)

for item in data:
    print(item['file_name'])

route-24226734-po-neve.gpx
route-23923148-kirillov-vologda.gpx
route-23922202-svir-vytegra.gpx
route-23922159-lipi-bor-vytegra.gpx
route-23922080-belozersk-lipii-bor.gpx
route-23922062-cherepovets-belozersk.gpx
route-23872309-vyborg-priozersk.gpx
route-23871404-ochen-strannyi-torzhok-likhoslavl.gpx
route-23833098-novgorod-staraia-russa.gpx
route-23766746-volkhov.gpx
route-23735851-aleksin-tarusa-serpukhov.gpx
route-23735746-mitliaevsko-nikolo-lenivets-kaluga.gpx
route-23287876-balabanovo-mozhaisk.gpx
route-22873154-kostroma-iaroslavl.gpx
route-22869365-valdai-uglovka.gpx
route-22869023-pushkinskie-gory-loknia.gpx
route-22869014-pskov-ostrov-pushkinskie-gory.gpx
route-22868739-pskov-pechory-izborsk.gpx
route-22693474-lianozovo-iauza-botanicheskii-sad.gpx
route-22686902-ostankino-likhobory.gpx
route-22683737-khoroshevo-serebrianyi-bor-fili.gpx
route-22625704-sokolniki-losinyi-ostrov.gpx
route-22625621-skolkovo-meshcherskii-park-peredelkino-odintsovo.gpx
route-22090769-belyi-gorodok-verbi

In [16]:
# STEP 5


import datetime
import random



def fprint(file, tab, s = '', end = '\n', tabsExist = True):
    st = ''
    for i in range(tab):
        if tabsExist:
            st += ''
            st += '\t'
    
    #end = ''
    file.write(st + s + end)

    

def getStyleColorId(state, color_scheme, color_thickness):
    #print(color)
    #return ''
    return 'strokeColor:' + color_scheme[state] + '_strokeWidth:' + str(color_thickness)
    

    
def genDescriptinForYnadexMapObject(id, name, date, href):
    description = ''
    description += name + ' '
    if date:
        description += '<br>'
        description += str(date) + ' '
        
    #description += '\n'    
    #description += 'id: ' + str(id)
    if href:
        description += '<br>'
        description += '<a href="' + href + '">' + href + '</a>'
    
    return description
    
    
def generateKmlFileForYandexMap(data = None, file_name = '', state_colors = None, title_map = '', description_map = ''):
    if not file_name or not data or not state_colors:
        return
    
    f = open('render-' + file_name,"w+")
    
    tab = 0
    thickness = 3

    fprint(f,tab,'<?xml version="1.0" encoding="UTF-8"?>\n')    
    fprint(f,tab,'<kml xmlns="http://www.opengis.net/kml/2.2">')  
    tab += 1

    fprint(f,tab,'<Document>') 
    tab += 1

    for state in state_colors:
        fprint(f,tab,'<Style id="' + getStyleColorId(state, state_colors, thickness) + '">')
        tab += 1
        fprint(f,tab,'<LineStyle>')
        tab += 1

        fprint(f,tab,'<color>' + state_colors[state] + '</color>')
        fprint(f,tab,'<width>' + str(thickness) + '</width>')

        tab -= 1
        fprint(f,tab,'</LineStyle>')

        tab -= 1
        fprint(f,tab,'</Style>')
    
    fprint(f,tab,'<name><![CDATA[' + title_map + ']]></name>')
    fprint(f,tab,'<description><![CDATA[' + description_map + ']]></description>')
        
    fprint(f,tab,'<ExtendedData>')
    tab += 1

    fprint(f,tab,'<Data name="creator">')
    tab += 1

    fprint(f,tab,'<value><![CDATA[Yandex Map Constructor]]></value>')

    tab -= 1
    fprint(f,tab,'</Data>')

    tab -= 1
    fprint(f,tab,'</ExtendedData>')

    fprint(f,tab,'<name><![CDATA[test]]></name>')
    fprint(f,tab,'<description><![CDATA[test]]></description>')

    fprint(f,tab,'<Folder>') 
    tab += 1

    for activity in data:   
        lat = []
        lon = []


        fprint(f,tab,'<Placemark>')
        tab += 1

        fprint(f,tab,'<name />')

        print(activity['file'])
        
        href = ''
        description_text = genDescriptinForYnadexMapObject(
            id, 
            name = activity['tracks'][0]['name'], 
            date = activity['time'], 
            href = activity['link']
        )
        
        fprint(f,tab,'<description><![CDATA[' + description_text + ']]></description>')

        fprint(f,tab,'<styleUrl>#' + getStyleColorId(activity['state'], state_colors, thickness) +  '</styleUrl>')

        fprint(f,tab,'<LineString>')
        tab += 1

        fprint(f,tab,'<coordinates>', '')

        for track in activity['tracks']:
            for segment in track['segments']:

                #print('len:' + str(len(segment['points'])))
                for point in segment['points'][:990]:
                    fprint(f,tab,str(point[1]) + ',' + str(point[0]) + ' ', '', False)

        fprint(f,tab,'</coordinates>', '\n', False)

        tab -= 1
        fprint(f,tab,'</LineString>')

        tab -= 1
        fprint(f,tab,'</Placemark>')

    tab -= 1
    fprint(f,tab,'</Folder>') 

    tab -= 1
    fprint(f,tab,'</Document>') 

    tab -= 1
    fprint(f,tab,'</kml>')  

    f.close()
    

    
    
    
    
    
    
    
    
    
    
# Change Some information below    
    
    
state_color_scheme = {
    'past_activity': 'e603ad1b',
    'past_route': 'e603ad1b',
    'current': 'e64345ed',
    'future': 'e6c97b17',
}


now_date = str(datetime.datetime.now()).split(' ')[0]

title = 'my Strava Bike Routes U-' + now_date
slug = slugify(title)

output_kml_file_name = slug + '.kml'


description_map  = 'past activities - green <br> this year activities - red <br>future routes - blue '



generateKmlFileForYandexMap(data, output_kml_file_name, state_color_scheme, title, description_map)

/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/route-22869014-pskov-ostrov-pushkinskie-gory.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/route-23735851-aleksin-tarusa-serpukhov.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/past-activity-21875514-rybinsk-romanov-last.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/activity-2702598567-2019-09-12-kimry-konakovo.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/route-17337486-staritsa-tver.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/route-17329026-ples-iurevets.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/activity-1197883696-2017-09-23-dnevnoi-velozaezd.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/route-17409938-petrozavodsk-kondopoga.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/route-15129981-pereslavl-pl

/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/route-22625704-sokolniki-losinyi-ostrov.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/route-23922159-lipi-bor-vytegra.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/route-18585257-zubtsov-staritsa.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/past-activity-21875525-romanov-iaroslavl-last.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/activity-1164565163-2017-09-02-utrennii-velozaezd.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/activity-2696542680-2019-09-10-kurovskoe-kolomna.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/route-22049242-verbilki-abramtsevo.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/activities/activity-1629672600-2018-06-10-tver-migalovo-domotkanovo-chupriianovo.gpx
/Users/andreylevin/Dropbox/git/mygithub/bike-routs/strava/acti

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import mplleaflet

import datetime
import random

import matplotlib.patches as mpatches

lat = []
lon = []

fig = plt.figure(facecolor = '0.05')
ax = plt.Axes(fig, [0., 0., 1., 1.], )
ax.set_aspect('equal')
ax.set_axis_off()
fig.add_axes(ax)




colors_red = [
    '#FFCC00',
    '#CC9933',
    '#FF6600',
    '#CC3300',
    '#CC3333',
    '#FF3366',
    '#993366',  
]

colors_no_red = [
    '#3300CC',
    '#0033FF',
    '#0099FF',
    '#003333',
    '#006633',
    '#00FF66',
    '#00CC00',
    '#669900',
    '#99CC00',
    '#666633',
]

color_current_year = 'crimson'
color_last_years = 'green'
color_future = 'mediumblue'


def setColorDepandOnTime(name, time):    
    color = color_last_years
    if time == None:
        if 'last' in name:
            return color_last_years
        
        return color_future
    
    if(activity['time'].year >= 2019):
        return color_current_year
    
    return color


for activity in data:
    #print(activity['time'].month)
    
    
    color_rout = setColorDepandOnTime(activity['tracks'][0]['name'], activity['time'])
        
    
    lat = []
    lon = []
    for track in activity['tracks']:
        for segment in track['segments']:
            for point in segment['points']:
                
                
                lat.append(point[0])
                lon.append(point[1])
                
    #plt.plot([lon[-1],lon[0]], [lat[-1],lat[0]], 'v', color=color_rout, markersize=12)
    scatter = plt.scatter([lon[-1],lon[0]], [lat[-1],lat[0]], marker='v', s = 300, c=color_rout, linewidths=2, edgecolors='w')
    scatter.set_urls(['http://www.bbc.co.uk/news', 'http://www.google.com', None])
    plt.plot(lon, lat, color = color_rout, lw = 3, alpha = 0.9)
    

#legends = []

#legends.append(mpatches.Patch(color=color_last_years, label='Past routes up to this year'))
#legends.append(mpatches.Patch(color=color_current_year, label='Completed routes this year'))
#legends.append(mpatches.Patch(color=color_future, label='Future routes'))
#plt.legend(handles=legends) 
    
    

mplleaflet.show()

    
#filename = data_path + '.png'
#plt.savefig(filename, facecolor = fig.get_facecolor(), bbox_inches='tight', pad_inches=0, dpi=300)

In [ ]:
# STEP 2


import math

data999points = 0

print('len data', len(data))

data999points = data.copy()


maxPoints = 998


print('start')
for activity in data999points:

    for track in activity['tracks']:
        for segment in track['segments']:
            
            
            
            #print(len(segment['points']), activity['tracks'][0]['name'])
            
            if len(segment['points']) < maxPoints:
                continue
            
            
            #print(len(segment['points']), activity['tracks'][0]['name'])
            
            p = segment['points']
            #print(len(p))
            
            
            
            lenpoints = len(p)

            trsh = 1e-12
            
            
            def makeTreshold(val):
                stp = 0.1
                
                val = 1
                for i in range(degs):
                    val *= stp
                    
                return val
                
                
            while(len(p) > maxPoints):
                n = -1
                while n < lenpoints - 2:
                    n += 1
                    #print(n, lenpoints)
                    if n == 0 or n == 1:
                        continue;

                    def metr(p1,p2):
                        return (p1[0] - p2[0]) * (p1[0] - p2[0]) + (p1[1] - p2[1]) * (p1[1] - p2[1])

                    trLong = metr(p[n-2],p[n])
                    #print('\t', trLong)
                    trSum = metr(p[n-2],p[n-1]) + metr(p[n-1],p[n])
                    #print('\t', trSum)
                    trDifr = math.fabs(trSum - trLong)
                    if trDifr < 0:
                        print('!\tMUNUS ')
                    trSmall = metr(p[n-1],p[n])

                    #print('\t', trDifr, trSmall)


                    if trDifr < trsh:
                        p.pop(n)
                        lenpoints -= 1
                        n -= 1     
                        
                trsh = trsh * 1.4

        
        
            
            print(trsh, len(p))




In [ ]:
# STEP 4

data999points = []
print(len(data999points))


data999points = data_future + data_last + data_this
        
print(len(data999points))

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
x = np.linspace(0, 3*np.pi, 500)
plt.plot(x, np.sin(x**2))
plt.title('A simple chirp')
plt.show()

In [ ]:
# import pandas as pd
import pandas as pd


lst = ['Geeks', 'For', 'Geeks', 'is', 
            'portal', 'for', 'Geeks']
 
    
df = pd.DataFrame(lst)
print(df[:2])


datadf = pd.DataFrame(data)
print(df[:2])

In [ ]:

def get_gpx_data(filename):
    gpx_file = open(filename, 'r')
    return gpxpy.parse(gpx_file)


dir_activities = 'activities'
subdir_this_year_activities = '2019'
subdir_past_activities = 'past'


data_2019 = get_gpx_data(dir_activities + '/' + subdir_this_year_activities)
data_past = get_gpx_data(dir_activities + '/' + subdir_this_year_activities)







In [ ]:
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import gpxpy

import mplleaflet


data_path = 'act'
data = [f for f in listdir(data_path) if isfile(join(data_path,f))]

lat = []
lon = []

fig = plt.figure(facecolor = '0.05')
ax = plt.Axes(fig, [0., 0., 1., 1.], )
ax.set_aspect('equal')
ax.set_axis_off()
fig.add_axes(ax)

for activity in data:
    gpx_filename = join(data_path,activity)
    gpx_file = open(gpx_filename, 'r')
    gpx = gpxpy.parse(gpx_file)

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                lat.append(point.latitude)
                lon.append(point.longitude)
    plt.plot(lon, lat, color = 'deepskyblue', lw = 0.2, alpha = 0.8)
    lat = []
    lon = []


mplleaflet.show()

#filename = data_path + '.png'
#plt.savefig(filename, facecolor = fig.get_facecolor(), bbox_inches='tight', pad_inches=0, dpi=300)

In [ ]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np

import mplleaflet

# Load up the geojson data
filename = 'data'+ '/' + 'track.geojson'
with open(filename) as f:
    gj = json.load(f)

# Grab the coordinates (longitude, latitude) from the features, which we
# know are Points
xy = np.array([feat['geometry']['coordinates'] for feat in gj['features'][::10]])

print(len(xy))

xy = xy[150:]

# Plot the path as red dots connected by a blue line
#plt.hold(True)
plt.plot(xy[:,0], xy[:,1], 'r')
plt.plot(xy[:,0], xy[:,1], 'b.')

xp = xy[-1:,0]
yp = xy[-1:,1]

print(xp, yp)

#plt.plot(xy[:,0], xy[:,1], 'r')

#plt.scatter(xp, yp, s=1000, c=None, marker = 'v',linewidths=10, verts=None, edgecolors='g', zorder=10, alpha=0)
plt.scatter(xp, yp, marker='v', s = 500, c='w', alpha=0.8, linewidths=2, edgecolors='orange')



mapfile = 'output' + '.html'
# Create the map. Save the file to basic_plot.html. _map.html is the default
# if 'path' is not specified
mplleaflet.display()